## Install Required Packages

In [30]:
%%bash
python3.11 -m pip install --upgrade pip
pip install numpy
pip install scipy
pip install keras
pip install tensorflow

## Imports

In [42]:
import numpy as np
from scipy import io
import os
import pickle
from pprint import pprint

# Import function to get covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

# Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

# Import decoder functions
from Neural_Decoding.decoders import LSTMDecoder


## User Inputs (Modify as Necessary)

In [32]:
binned_dir = "Directions_Binned" # Name of directory that will store binned results

bins_before = 1 # How many bins of neural data prior to the output are used for decoding (fewer bins is faster)
bins_current = 1 # Whether to use concurrent time bin of neural data
bins_after = 1 # How many bins of neural data after the output are used for decoding (fewer bins is faster)

training_range = [0, 0.85] # Range of training data
testing_range = [0.85, 1] # Range of testing data

## LSTM

In [ ]:
# Get all .pickle files
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, binned_dir)
files = next(os.walk(data_dir))[2]

results = {}

for f in files:
    dataset = f.split('.')[0]
    print('dataset: ' + dataset)
    with open(data_dir + "/" + f, 'rb') as res:
        neural_data, pos_binned = pickle.load(res,encoding='latin1')

    X = get_spikes_with_history(neural_data, bins_before, bins_after, bins_current)
    y = pos_binned

    num_examples = X.shape[0]
    training_set = np.arange(int(np.round(training_range[0] * num_examples)) + bins_before, int(np.round(training_range[1] * num_examples)) - bins_after)
    testing_set = np.arange(int(np.round(testing_range[0] * num_examples)) + bins_before, int(np.round(testing_range[1] * num_examples)) - bins_after)

    # Get training data
    X_train = X[training_set, :, :]
    y_train = y[training_set, :]

    # Get testing data
    X_test = X[testing_set, :, :]
    y_test = y[testing_set, :]

    # Z-score "X" inputs. 
    X_train_mean = np.nanmean(X_train, axis=0)
    X_train_std = np.nanstd(X_train, axis=0)
    X_train = np.divide((X_train - X_train_mean), X_train_std, where=X_train_std[1] != 0)
    X_test = np.divide((X_test - X_train_mean), X_train_std, where=X_train_std[1] != 0)


    # Zero-center outputs
    y_train_mean = np.nanmean(y_train, axis=0)
    y_train = y_train - y_train_mean
    y_test = y_test - y_train_mean

    # Declare model
    model_lstm = LSTMDecoder(units=400, dropout=0, num_epochs=20, verbose=0)

    # Fit model
    model_lstm.fit(X_train, y_train)

    # Get predictions
    y_predicted_lstm = model_lstm.predict(X_test)

    # Get metric of fit
    rhos_lstm = get_rho(y_test, y_predicted_lstm)
    print('R2:', rhos_lstm ** 2)

    results[dataset] = rhos_lstm ** 2

In [55]:
# pprint(results)

m1s = {k : results[k] for k in results if "M1" in k}
s1s = {k : results[k] for k in results if "S1" in k}
rys = {k : results[k] for k in results if "Ry" in k}
yes = {k : results[k] for k in results if "Ye" in k}
drs = {k : results[k] for k in results if "Dr" in k}
fds = {k : results[k] for k in results if "Fd" in k}
cons = {k : results[k] for k in results if "Con" in k}
nbs = {k : results[k] for k in results if "NB" in k}
pprint(nbs)

{'RyDrNB_M1': array([0.94449533]),
 'RyDrNB_S1': array([0.00240857]),
 'RyFdNB_M1': array([0.82546535]),
 'RyFdNB_S1': array([0.0129843]),
 'YeDrNB_M1': array([0.21815668]),
 'YeDrNB_S1': array([0.00146558]),
 'YeFdNB_M1': array([0.71520104]),
 'YeFdNB_S1': array([0.1302585])}
